In [28]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
# above code just loads the model we prepared to check the masks
import numpy as np
import cv2
import os

In [31]:
prototxtPath=os.path.sep.join([r'/home/aayush/Desktop/Coding/Python/Face_detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'/home/aayush/Desktop/Coding/Python/Face_detector','res10_300x300_ssd_iter_140000.caffemodel'])
#we are not use harcasscade using caffemodel as it is much more better than harcasscade


In [33]:
prototxtPath

'/home/aayush/Desktop/Coding/Python/Face_detector/deploy.prototxt'

In [35]:
weightsPath
#above are two mainmiles which detect the face 
#it is Single short Detector

'/home/aayush/Desktop/Coding/Python/Face_detector/res10_300x300_ssd_iter_140000.caffemodel'

In [37]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)
#


In [39]:
#Loading our model
model=load_model(r'/home/aayush/Desktop/Coding/Python/mobilenet_v2.model')

In [41]:
#loading the image to be tested
image=cv2.imread(r'/home/aayush/Desktop/Coding/Python/wm2.jpeg')
#imread takes an image and convert into array

In [43]:
image

array([[[232, 223, 214],
        [232, 223, 214],
        [232, 223, 214],
        ...,
        [ 62,  89,  99],
        [ 68,  93, 103],
        [ 78, 103, 113]],

       [[232, 223, 214],
        [232, 223, 214],
        [232, 223, 214],
        ...,
        [ 67,  96, 105],
        [ 67,  94, 104],
        [ 66,  93, 103]],

       [[232, 223, 214],
        [232, 223, 214],
        [232, 223, 214],
        ...,
        [ 63,  93, 104],
        [ 68,  96, 107],
        [ 66,  94, 105]],

       ...,

       [[200, 189, 181],
        [199, 188, 180],
        [199, 188, 180],
        ...,
        [151, 148, 140],
        [151, 148, 140],
        [152, 149, 141]],

       [[199, 188, 180],
        [199, 188, 180],
        [198, 187, 179],
        ...,
        [152, 149, 141],
        [153, 150, 142],
        [153, 150, 142]],

       [[199, 188, 180],
        [199, 188, 180],
        [198, 187, 179],
        ...,
        [153, 150, 142],
        [154, 151, 143],
        [155, 152, 144]]

In [45]:
image.shape
#below shows the size of image and 3 is RGB channel

(668, 1000, 3)

In [47]:
(h,w)=image.shape[:2]
#we need 1280 and 720 i.e height and weight of image 

In [49]:
(h,w)

(668, 1000)

In [51]:
#Preprocessing it 
#Resize the image
#image,1.0:-1.0 is an scaling factor and we mentioned here that we are not scaling it
#here 104.0,117.0,123.0 is the colour channel
#blob takes into in BGR not RGB so this function by defaukts inputs in BGR
blob=cv2.dnn.blobFromImage(image,1.0,(300, 300),(104.0,177.0,123.0))

In [53]:
blob
#images getting Scaled down annd resized the image

array([[[[128., 127., 128., ...,  37., -44., -37.],
         [128., 127., 128., ...,  80.,   6., -43.],
         [128., 127., 128., ...,  95.,  69., -25.],
         ...,
         [ 95.,  93.,  91., ...,  39.,  39.,  41.],
         [ 95.,  92.,  91., ...,  47.,  47.,  47.],
         [ 95.,  92.,  91., ...,  47.,  49.,  49.]],

        [[ 46.,  45.,  44., ...,  -2., -83., -84.],
         [ 46.,  45.,  44., ...,  36., -32., -85.],
         [ 46.,  45.,  44., ...,  45.,  29., -63.],
         ...,
         [ 11.,   9.,   7., ..., -37., -36., -35.],
         [ 11.,   8.,   7., ..., -31., -29., -29.],
         [ 11.,   8.,   7., ..., -29., -27., -27.]],

        [[ 91.,  90.,  90., ...,  67., -20., -20.],
         [ 91.,  90.,  90., ..., 109.,  35., -20.],
         [ 91.,  90.,  90., ..., 116.,  99.,   5.],
         ...,
         [ 57.,  55.,  53., ...,   8.,   9.,  11.],
         [ 57.,  54.,  53., ...,  14.,  17.,  17.],
         [ 57.,  54.,  53., ...,  17.,  19.,  19.]]]], dtype=float32)

In [55]:
blob.shape
#its in $D

(1, 3, 300, 300)

In [57]:
net.setInput(blob)
detections=net.forward()

In [58]:
detections
#these arrays are indicating the face region of the image
#that is our face detector model detected all the faces now we shall check for the mask

array([[[[0.        , 1.        , 0.97395396, ..., 0.28215754,
          0.6688398 , 0.59488446],
         [0.        , 1.        , 0.17017119, ..., 0.5486332 ,
          0.4041294 , 0.6962155 ],
         [0.        , 1.        , 0.12901571, ..., 0.512806  ,
          0.4563606 , 0.6793694 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [59]:
detections.shape
#this 200 is the shape portion

(1, 1, 200, 7)

In [60]:
#now here we shall be detecting the face with mask or without mask

In [61]:
#loop over the detections


#detections[2] is the face portion
for i in range(0,detections.shape[2]):

    confidence=detections[0,0,i,2]
    
    #confidence is the accuracy level which checks weather the person is wearing mask       or not
    
    if confidence>0.5:
        #we need the X,Y coordinates
        #creating the box portion indication the mask and face portion
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])


        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        #that is the box indicating the mask or frame portion shouldn't go outside the frame section
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        

        face=image[startY:endY, startX:endX]
        #blob took input in BGR so converting it to RGB
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        #we trained our model to detect mask for size 224,224
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        #green color if mask else in red color BGR
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        #till 2 places of decimal
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.40,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)

In [56]:
#shows the ouput screen
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()